In [1]:
import scrapy
from urllib.request import urlopen #to open the urls that the dois are put into
import json
import requests
import pprint

#don't think I'm currently using these two things
from scrapy.http import TextResponse
from scrapy.utils.markup import remove_tags


#for the ArticleItem section
from scrapy.item import Item, Field
from scrapy.selector import Selector
from scrapy.spiders import Spider

from scrapy import Spider #for the spider ArticleSpider

#to run the spider in Jupyter notebook, have to restart the kernel each time to run it
from scrapy.settings import Settings
from scrapy.crawler import CrawlerProcess

In [2]:
#doi_list = [ACS, Elsevier, Springer, Aiche Journal]
#automatically decides which journal a doi is for, then it will follow a certain set of instructions
doi_list = ["10.1021/ja302991b", "10.1016/j.micromeso.2012.01.033", "10.1007/s10450-012-9423-1", "10.1002/aic.690470520", "10.1007/s10450-013-9527-2"]
for d in doi_list:
    test_url = 'http://dx.doi.org/{0}'.format(d)
    
    headers = {'Accept': 'application/citeproc+json'}
    bib_info = json.loads(requests.get(test_url, headers=headers).content)
    if bib_info['publisher'] == 'American Chemical Society (ACS)':
        
        doi_acs = bib_info.get('DOI')
        full_url_acs = 'http://pubs.acs.org/doi/full/{0}'.format(doi_acs)
        print(full_url_acs)
        response_acs = urlopen(full_url_acs)
        content_acs = response_acs.read()
 
        
    elif bib_info['publisher'] == 'Elsevier BV':
        
        doi_el = bib_info.get('DOI')
        full_url_el = 'http://dx.doi.org/{0}'.format(doi_el)
        print(full_url_el)
        response_el = urlopen(full_url_el)
        content_el = response_el.read()
        
    elif bib_info['publisher'] == 'Springer Nature':
        
        doi_spr = bib_info.get('DOI')
        full_url_spr = 'http://link.springer.com/article/{0}'.format(doi_spr)
        print(full_url_spr)
        response_spr = urlopen(full_url_spr)
        content_spr = response_spr.read()
        
    else:
        print('wrong publisher')

http://pubs.acs.org/doi/full/10.1021/ja302991b
http://dx.doi.org/10.1016/j.micromeso.2012.01.033
http://link.springer.com/article/10.1007/s10450-012-9423-1
wrong publisher
http://link.springer.com/article/10.1007/s10450-013-9527-2


In [4]:
#In the scrapy project project_name/items.py
from scrapy.item import Item, Field
from scrapy.selector import Selector
from scrapy.spiders import Spider

class ArticleItem(Item):
    title = Field()
    authors = Field()
    doi = Field()
    abstract = Field()
    text = Field()
    figures = Field()

In [ ]:
#In scrapy project in the spiders/ directory
class ArticleSpider(scrapy.Spider):
    name = 'ArticleSpider'
    allowed_domains = ["http://pubs.acs.org/"]
    start_urls = ["http://pubs.acs.org/doi/full/10.1021/ja302991b"]
    
    def parse(self, response):
        item = ArticleItem()
        item['title'] = response.xpath('//span[@class="hlFld-Title"]/text()').extract()
        item['authors'] = response.xpath('//div[@id="authors"]/text()').extract()
        item['doi'] = response.xpath('//div[@id="doi"]/text()').extract()
        item['abstract'] = response.xpath('//p[@class="articleBody_abstractText"]/text()').extract
        item['text'] = response.xpath('//div[@class="hlFld-Fulltext"]').extract()
        item['figures'] = response.xpath('//img[@alt="figure"]').extract()
        yield item

In [7]:
class JsonItemExporter(JsonLinesItemExporter):
    
    def __init__(self, file, **kwargs):
        self._configure(kwargs)
        self.file = file
        self.encoder = json.JSONEncoder(**kwargs)
        self.first_item = True
    
    def start_exporting(self):
        self.file.write("[")
        
    def finish_exporting(self):
        self.file.write("]")
    
    def export_item(self, item):
        if self.firsrt_item:
            self.first_item = False
        else:
            self.file.write(',\n')
        itemdict = dict(self._get_serialized_fields(item))
        self.file.write(self.encoder.encode(itemdict))

NameError: name 'JsonLinesItemExporter' is not defined

In [5]:
from scrapy.settings import Settings
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
process.crawl(ArticleSpider)
process.start()

2017-11-07 08:04:31 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-11-07 08:04:31 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-11-07 08:04:31 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-11-07 08:04:31 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

2017-11-07 08:04:35 [scrapy.core.engine] INFO: Closing spider (finished)
2017-11-07 08:04:35 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 2107,
 'downloader/request_count': 3,
 'downloader/request_method_count/GET': 3,
 'downloader/response_bytes': 191384,
 'downloader/response_count': 3,
 'downloader/response_status_count/200': 1,
 'downloader/response_status_count/302': 2,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 11, 7, 13, 4, 35, 58301),
 'item_scraped_count': 1,
 'log_count/DEBUG': 5,
 'log_count/INFO': 7,
 'memusage/max': 70230016,
 'memusage/startup': 70230016,
 'response_received_count': 1,
 'scheduler/dequeued': 3,
 'scheduler/dequeued/memory': 3,
 'scheduler/enqueued': 3,
 'scheduler/enqueued/memory': 3,
 'start_time': datetime.datetime(2017, 11, 7, 13, 4, 31, 271243)}
2017-11-07 08:04:35 [scrapy.core.engine] INFO: Spider closed (finished)


In [3]:
class ArticleSpiderEl(scrapy.Spider):
    name = 'ArticleSpiderEl'
    allowed_domains = ["https://www.elsevier.com/"]
    start_urls = ["http://dx.doi.org/10.1016/j.micromeso.2012.01.033"]
    
    def parse(self, response):
        item = ArticleItem()
        item['title'] = response.xpath('//span[@class="title-text"]/text()').extract()
        item['authors'] = response.xpath('//div[@class="author-group"]').extract()
        item['doi'] = response.xpath('//a[@class="DoiLink"]/text()').extract()
        item['abstract'] = response.xpath('//div[@class="abstract author"]/text()').extract
        item['text'] = response.xpath('//div[@class="Body"]').extract()
        item['figures'] = response.xpath('//figure').extract()
        yield item

In [5]:
from scrapy.settings import Settings
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
process.crawl(ArticleSpiderEl)
process.start()

2017-11-07 08:06:45 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-11-07 08:06:45 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-11-07 08:06:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-11-07 08:06:45 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

In [5]:
class ArticleSpiderSpr(scrapy.Spider):
    name = 'ArticleSpiderSpr'
    allowed_domains = ["https://link.springer.com/journal/10450"]
    start_urls = ["http://link.springer.com/article/10.1007/s10450-012-9423-1"]
    
    def parse(self, response):
        item = ArticleItem()
        item['title'] = response.xpath('//h1[@class="ArticleTitle"]/text()').extract()
        item['authors'] = response.xpath('//div[@class="authors_list"]').extract()
        item['doi'] = response.xpath('//span[@id="doi-url"]/text()').extract()
        item['abstract'] = response.xpath('//section[@class="Abstract"]/text()').extract
        item['text'] = response.xpath('//div[@id="body"]').extract()
        item['figures'] = response.xpath('//div[@class="MediaObject"]').extract()
        yield item

In [6]:
from scrapy.settings import Settings
from scrapy.crawler import CrawlerProcess

process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'})
process.crawl(ArticleSpiderSpr)
process.start()

2017-11-07 08:40:24 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-11-07 08:40:24 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-11-07 08:40:24 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-11-07 08:40:24 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

2017-11-07 08:40:25 [scrapy.core.engine] INFO: Closing spider (finished)
2017-11-07 08:40:25 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 530,
 'downloader/request_count': 2,
 'downloader/request_method_count/GET': 2,
 'downloader/response_bytes': 32546,
 'downloader/response_count': 2,
 'downloader/response_status_count/200': 1,
 'downloader/response_status_count/301': 1,
 'finish_reason': 'finished',
 'finish_time': datetime.datetime(2017, 11, 7, 13, 40, 25, 362848),
 'item_scraped_count': 1,
 'log_count/DEBUG': 4,
 'log_count/INFO': 7,
 'memusage/max': 70410240,
 'memusage/startup': 70410240,
 'response_received_count': 1,
 'scheduler/dequeued': 2,
 'scheduler/dequeued/memory': 2,
 'scheduler/enqueued': 2,
 'scheduler/enqueued/memory': 2,
 'start_time': datetime.datetime(2017, 11, 7, 13, 40, 24, 416875)}
2017-11-07 08:40:25 [scrapy.core.engine] INFO: Spider closed (finished)
